In [1]:
import pandas as pd

In [24]:
'Load data'
loan_data = pd.read_csv('01Exercise1.csv')
loan_prep = loan_data.copy() #keeps original data intact
loan_prep.info()
loan_prep.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 6 columns):
gender     601 non-null object
married    611 non-null object
ch         564 non-null float64
income     614 non-null int64
loanamt    592 non-null float64
status     614 non-null object
dtypes: float64(2), int64(1), object(3)
memory usage: 28.9+ KB


,ch,income,loanamt
count,564.000000,614.000000,592.000000
mean,0.842199,5403.459283,146.412162
std,0.364878,6109.041673,85.587325
min,0.000000,150.000000,9.000000
25%,1.000000,2877.500000,100.000000
50%,1.000000,3812.500000,128.000000
75%,1.000000,5795.000000,168.000000
max,1.000000,81000.000000,700.000000


In [18]:
'Clean the data'
loan_prep.isnull().sum(axis=0) #we see some null values
loan_prep = loan_prep.dropna() #drop rows with missing values
loan_data.head(2)

,gender,married,ch,income,loanamt,status
0,Male,No,1.0,5849,NaN,Y
1,Male,Yes,1.0,4583,128.0,N


In [19]:
"Note that gender is not likely to predict loan status, so we may drop it"
#loan_prep.drop(['gender'],axis=1) to drop the gender column
loan_prep.head(2)

,married,ch,income,loanamt,status
1,Yes,1.0,4583,128.0,N
2,Yes,1.0,3000,66.0,Y


In [64]:
'Create dummy variables'
loan_prep.dtypes #check types of each value
loan_prep = pd.get_dummies(loan_prep,drop_first=True) #obtain dummy variables

In [65]:
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
loan_prep['income'] = scalar.fit_transform(loan_prep[['income']])
loan_prep['loanamt'] = scalar.fit_transform(loan_prep[['loanamt']])

In [66]:
'Create Variables for analysis'
y = loan_prep[['status_Y']]
x = loan_prep.drop(['status_Y'], axis=1)

In [67]:
'Split the data to obtain test and training data'
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = \
train_test_split(x,y,test_size=0.3, random_state=1234, stratify=y)

# Stratification helps ensure that the test and train data contains diversity in the y-value. In the case of loan approvals, we have two outcomes 'yes' and 'no'. We want to make sure that the test/train data has a certain number of yes values and a certain number of no values.

In [76]:
'Build logistic regressor'
import numpy as np
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
'Training data had null and inf values, this replaces them with the mean'
x_train[x_train==np.inf]=np.nan
x_train.fillna(x_train.mean(), inplace=True)
lr.fit(x_train,y_train)

C:\Users\clyde\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\clyde\Anaconda3\lib\site-packages\pandas\core\frame.py:3515: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)
C:\Users\clyde\Anaconda3\lib\site-packages\pandas\core\generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [78]:
x_test[x_test==np.inf]=np.nan
x_test.fillna(x_test.mean(), inplace=True)
y_pred = lr.predict(x_test)

C:\Users\clyde\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\clyde\Anaconda3\lib\site-packages\pandas\core\frame.py:3515: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)
C:\Users\clyde\Anaconda3\lib\site-packages\pandas\core\generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_gui

In [79]:
'Use confusion matrix to examine test and predicted values'
from sklearn.metrics import confusion_matrix
conf = confusion_matrix(y_test,y_pred)

In [80]:
conf #more false negatives, work needed on the model

array([[ 18,  40],
       [  3, 124]], dtype=int64)

In [81]:
'Examine accuary'
lr.score(x_test,y_test)
'The score obtained shows that another model could be used'

0.7675675675675676

In [89]:
from sklearn.metrics import accuracy_score
score2 = accuracy_score(y_test,y_pred)
y_prob = lr.predict_proba(x_test)[:,1]
score2

0.7675675675675676

In [91]:
'Probability values'
y_new_pred = []
threshold = 0.8 #probability threshold

for i in range(0,len(y_prob)):
    if y_prob[i]>threshold:
        y_new_pred.append(1)
    else:
        y_new_pred.append(0)


In [97]:
from sklearn.metrics import classification_report
conf2 = confusion_matrix(y_test,y_new_pred)
score2 = lr.score(x_test,y_test)
cr2 = classification_report(y_test,y_new_pred)
print(conf2, '\n',score2, '\n',cr2)

[[46 12]
 [61 66]] 
 0.7675675675675676 
               precision    recall  f1-score   support

           0       0.43      0.79      0.56        58
           1       0.85      0.52      0.64       127

    accuracy                           0.61       185
   macro avg       0.64      0.66      0.60       185
weighted avg       0.72      0.61      0.62       185

